In [1]:
# import libraries
import pandas as pd
import numpy as np
import helpers
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# set plot theme
plt.style.use('ggplot')

# set dataframe display 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 10000)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 10000)

In [2]:
# import training data

train_df = pd.read_csv('../data/train.csv')

train_df = helpers.clean_headers(train_df)


print(train_df.head())

  sex  length  diameter  height     weight  shucked_weight  viscera_weight  shell_weight  age
0   I  1.5250    1.1750  0.3750  28.973189       12.728926        6.647958      8.348928    9
1   I  1.1000    0.8250  0.2750  10.418441        4.521745        2.324659      3.401940    8
2   M  1.3875    1.1125  0.3750  24.777463       11.339800        5.556502      6.662133    9
3   F  1.7000    1.4125  0.5000  50.660556       20.354941       10.991839     14.996885   11
4   I  1.2500    1.0125  0.3375  23.289114       11.977664        4.507570      5.953395    8


In [3]:
# normalize numerical variables and one hot encode categorical variables

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['length', 'diameter', 'height', 'weight','shucked_weight', 'viscera_weight', 'shell_weight']),
        ('cat', OneHotEncoder(), ['sex'])
    ]
)

# apply transformations
scaled_data = preprocessor.fit_transform(train_df)

# get column headers
column_names = preprocessor.get_feature_names_out()

# Convert transformed data back to Dataframe
scaled_df = pd.DataFrame(scaled_data, columns=column_names)

scaled_df['age'] = train_df['age'] # add target variable to df

print(scaled_df.head()) # debug

   num__length  num__diameter  num__height  num__weight  num__shucked_weight  num__viscera_weight  num__shell_weight  cat__sex_F  cat__sex_I  cat__sex_M  age
0     0.721238       0.633982     0.292400     0.441804             0.467188             0.569186           0.453376         0.0         1.0         0.0    9
1    -0.755712      -0.840356    -0.794163    -1.025198            -0.993688            -0.978880          -0.926788         0.0         1.0         0.0    8
2     0.243401       0.370707     0.292400     0.110076             0.219924             0.178363          -0.017224         0.0         0.0         1.0    9
3     1.329394       1.634426     1.650603     2.156483             1.824616             2.124622           2.308095         1.0         0.0         0.0   11
4    -0.234435      -0.050532    -0.115061    -0.007598             0.333464            -0.197233          -0.214955         0.0         1.0         0.0    8


In [4]:
# Checking for MULTICOLLINEARITY

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Add a constant term for the intercept
X_with_constant = add_constant(train_df.iloc[:,1:])

# Calculate VIF for each feature
vif = pd.DataFrame()
vif["Variable"] = X_with_constant.columns
vif["VIF"] = [variance_inflation_factor(X_with_constant.values, i) for i in range(X_with_constant.shape[1])]

print(vif)

         Variable        VIF
0           const  75.590550
1          length  50.036365
2        diameter  52.355138
3          height   7.811277
4          weight  78.053036
5  shucked_weight  25.301892
6  viscera_weight  17.968955
7    shell_weight  20.997071
8             age   2.170267


In [5]:
### Feature Engineering

train_df['heightXweight'] = train_df['height'] * train_df['weight']
train_df['lenXdiameter'] = train_df['length'] * train_df['diameter']

train_df = train_df.drop(columns = ['length', 'diameter', 'height', 'weight'
                                    ,'shucked_weight', 'viscera_weight'])

print(train_df.head())

  sex  shell_weight  age  heightXweight  lenXdiameter
0   I      8.348928    9      10.864946      1.791875
1   I      3.401940    8       2.865071      0.907500
2   M      6.662133    9       9.291549      1.543594
3   F     14.996885   11      25.330278      2.401250
4   I      5.953395    8       7.860076      1.265625


In [6]:
# Checking for MULTICOLLINEARITY

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Add a constant term for the intercept
X_with_constant = add_constant(train_df.iloc[:,1:])

# Calculate VIF for each feature
vif = pd.DataFrame()
vif["Variable"] = X_with_constant.columns
vif["VIF"] = [variance_inflation_factor(X_with_constant.values, i) for i in range(X_with_constant.shape[1])]

print(vif)

        Variable        VIF
0          const  20.062251
1   shell_weight  18.510732
2            age   1.874967
3  heightXweight  12.763791
4   lenXdiameter  10.712486


Considering that there is still high VIF, which indicates multicollinearity, we will apply PCA to reduce the dimensionality.

In [7]:
# Seperating the target and response variables
y = scaled_df['age'] # target variable
X = scaled_df.iloc[:, :-1] # independent variable

# Models

#### Linear Regression

In [8]:
# Fit Linear Regression

linreg = LinearRegression()

lin_reg_cv = helpers.get_cv_results(linreg, X, y, 5)


Cross Validation Results:
fit_time [0.00956583 0.01857829 0.00779581 0.0064919  0.00660205]
score_time [0.0027411  0.00439072 0.00159216 0.00095725 0.00093198]
test_neg_mean_absolute_error [-1.47402493 -1.49101239 -1.46569384 -1.49755124 -1.48386011]
train_neg_mean_absolute_error [-1.48501352 -1.47994682 -1.48551906 -1.47837694 -1.48208908]
test_neg_mean_squared_error [-4.49574569 -4.57531402 -4.4214374  -4.66123975 -4.49847563]
train_neg_mean_squared_error [-4.53736852 -4.51706078 -4.55543444 -4.49543106 -4.53621669]
test_r2 [0.55185068 0.55245599 0.55929887 0.54206462 0.5474973 ]
train_r2 [0.55049563 0.55037521 0.5486953  0.55302012 0.55161091]


Looking at the cross validations scores from Linear Regression Model, we can see that the values of MAE and MSE are relatively close to each other. 

However, the R2 value only shows that the model is able to predict about 55% of the variation. It is likely that the model assumption for linear regression is broken. 

#### XGBoost

In [10]:
xgb_model = xgb.XGBRegressor()

cv_results = helpers.get_cv_results(xgb_model, X, y, 5)



Cross Validation Results:
fit_time [0.18053102 0.15645289 0.16443181 0.15467    0.1496253 ]
score_time [0.00559592 0.00598812 0.00554609 0.00549722 0.00565505]
test_neg_mean_absolute_error [-1.41469005 -1.42097167 -1.40928411 -1.4318559  -1.42525562]
train_neg_mean_absolute_error [-1.25630621 -1.2558969  -1.26195309 -1.25189227 -1.25302246]
test_neg_mean_squared_error [-4.30115113 -4.30425002 -4.22842024 -4.35225103 -4.29872033]
train_neg_mean_squared_error [-3.23069295 -3.24163705 -3.27716427 -3.21106399 -3.22181742]
test_r2 [0.57124841 0.57897067 0.57853764 0.57242072 0.56759071]
train_r2 [0.67994434 0.6773299  0.6753329  0.6807245  0.68153459]
